# SQL Reference Notebook

General pointers:
- a semicolon denotes the end of a query (optional)
- SQL assumes if you divide an integer by an integer, you want an integer back, so be cognisant of decimal points

## Common Keywords

- DISTINCT - returns unique values
- COUNT - returns the number of records
- WHERE - allows filtering (text and numeric), always after FROM
- AND/OR - logical chains
- BETWEEN - upper and lower limit
- IN - specify multiple values in a WHERE clause
- NULL - represents missing or unknown value
- LIKE - search for text pattern
- AS - aliasing
- ORDER BY - sorts results by values in a column  (default ascending)
- GROUP BY - groups results by one or more columns
- JOIN - combine tables on common columns (keys)
- USING - when joining on columns with the same name, sub `USING(id)` for `ON`
- CASE - allows multiple `IF-THEN-ELSE` statements in a simplified way (calculated column)
- INTO - allows placement of query results into a new table

## Aggregate Functions

- AVG - average
- MAX - max
- SUM - sum
- MIN - min

## COUNT
`COUNT(*)` returns the number of rows in a table, `COUNT(column_name)` returns the number of non-missing values in the selected column.

In [ ]:
# example 1
SELECT COUNT(*)
FROM people;

# example 2
SELECT COUNT(birthdate)
FROM people;

# example 3
SELECT COUNT(DISTINCT birthdate)
FROM people;

## WHERE

Note, when combining `AND` and `OR` be sure to enclose the individual clauses in parentheses.

In [ ]:
# example 1
SELECT title
FROM films
WHERE release_year > 1994
AND release_year < 2000;

# example 2
SELECT *
FROM films
WHERE language = 'Spanish'
AND release_year > 2000;

# example 3
SELECT title
FROM films
WHERE release_year = 1994
OR release_year = 2000;

# example 4
SELECT title
FROM films
WHERE (release_year = 1994 OR release_year = 1995)
AND (certification = 'PG' OR certification = 'R');

## BETWEEN

Note that `BETWEEN` is inclusive.

In [ ]:
# example 1
SELECT title
FROM films
WHERE release_year
BETWEEN 1994 AND 2000;

# example 2
SELECT name
FROM kids
WHERE age BETWEEN 2 AND 12
AND nationality = 'USA';

## IN

In [ ]:
# example 1
SELECT name
FROM kids
WHERE age IN (2, 4, 6, 8, 10);

## NULL, IS NULL, IS NOT NULL

In [ ]:
# example 1
SELECT COUNT(*)
FROM people
WHERE birthdate IS NULL;

# example 2
SELECT name
FROM people
WHERE birthdate IS NOT NULL;

## LIKE, NOT LIKE

Here, `%` represents a wildcard matching any zero, one, or many characters and `_` represents a wildcard matching a *single* character.

In [ ]:
# example 1
SELECT name
FROM companies
WHERE name LIKE 'Data%';

# example 2
SELECT name
FROM companies
WHERE name LIKE 'DataC_mp';

## Aggregate Functions

In [ ]:
# example 1
SELECT AVG(budget)
FROM films;

# example 2
SELECT MAX(budget)
FROM films;

# example 3
SELECT SUM(budget)
FROM films
WHERE release_year >= 2010;

## AS and arithmetic

In [ ]:
# example 1
SELECT MAX(budget) AS max_budget,
       MAX(duration) AS max_duration
FROM films;

# example 2
SELECT
title, (gross - budget) AS net_profit
FROM
films;

# example 3
SELECT
title, (duration/60.0) AS duration_hours
FROM
films;

## ORDER BY

In [ ]:
# example 1 - DESC = descending
SELECT title
FROM films
ORDER BY release_year DESC;

# example 2
SELECT title
FROM films
WHERE (release_year = 2000 OR release_year = 2012)
ORDER BY release_year;

# example 3
SELECT birthdate, name
FROM people
ORDER BY birthdate, name;

## GROUP BY

As with pandas, `GROUP BY` is commonly used in conjunction with an aggregate function such as `COUNT()` or `MAX()` and is always applied after the `FROM` clause. Also note that you need to `GROUP BY` all fields that aren't aggregated in your `SELECT` clause.

In [ ]:
# example 1
SELECT sex, count(*)
FROM employees
GROUP BY sex;

# example 2
# see Left JOIN, example 2

## JOIN

`JOIN` is a powerful tool to combine data from different tables. There are several different options that determine how the data is combined.

Note that `LEFT JOIN`, `RIGHT JOIN`, and `FULL JOIN` are all classified as outer joins.

### Inner JOIN

Select all records from Table A and Table B where join condition is met.
<p></p><img src="images/inner_join.png"><p></p>

In [ ]:
# example 1
SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id;

# example 2 - multiple inner JOIN
SELECT *
FROM left_table
INNER JOIN right_table
ON left_table.id = right_table.id
INNER JOIN table1
ON left_table.id = table1.id;

# example 3 - also adding calculated field
SELECT p1.country_code, 
       p1.size AS size2010,
       p2.size AS size2015,
       (p2.size-p1.size)/p1.size * 100.0 AS growth_perc
FROM populations AS p1
INNER JOIN populations AS p2
ON  p1.country_code = p2.country_code
AND p1.year = p2.year-5;

### Left JOIN

Select all records from Talbe A along with records from Table B for which the join condition is met (if at all).
<p></p><img src="images/left_join.png"><p></p>

In [ ]:
# example 1
SELECT c.name AS country, local_name, l.name AS language, percent
FROM countries AS c
LEFT JOIN languages AS l
ON c.code = l.code
ORDER BY country DESC;

# example 2
SELECT region, AVG(gdp_percapita) AS avg_gdp
FROM countries AS c
LEFT JOIN economies AS e
USING(code)
WHERE year = 2010
GROUP BY c.region
ORDER BY avg_gdp;

# example 3 - see Right JOIN example 1 for equivalent
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM cities
LEFT JOIN countries
ON cities.country_code = countries.code
LEFT JOIN languages
ON countries.code = languages.code
ORDER BY city, language;

### Right JOIN

Select all records from Table B, along with records from Table A for which the join condition is met (if at all). Right joins are not as common as left joins, mostly because you can always write a right join as a left join.
<p></p><img src="images/right_join.png"><p></p>

In [ ]:
# example 1 - see Left JOIN example 3 for equivalent
SELECT cities.name AS city, urbanarea_pop, countries.name AS country,
       indep_year, languages.name AS language, percent
FROM languages
RIGHT JOIN countries
ON languages.code = countries.code
RIGHT JOIN cities
ON countries.code = cities.country_code
ORDER BY city, language;

### Full JOIN

Select all records from Table A and Table B, regardless of whether the join condition is met or not.
<p></p><img src="images/full_join.png"><p></p>

In [ ]:
# example 1
SELECT name AS country, code, region, basic_unit
FROM countries AS c1
FULL JOIN currencies AS c2
USING (code)
WHERE c1.region = 'North America' OR c1.region IS NULL
ORDER BY region;

# example 2
SELECT c.name, code, l.name AS language
FROM languages AS l
FULL JOIN countries AS c
USING (code)
WHERE c.name LIKE 'V%' OR c.name IS NULL
ORDER BY name;

# example 3 - double full join
SELECT c1.name AS country, region, l.name AS language,
       c2.basic_unit, c2.frac_unit
FROM countries AS c1
FULL JOIN languages AS l
USING (code)
FULL JOIN currencies AS c2
USING (code)
WHERE c1.region LIKE 'M%esia';

### Cross JOIN

Selects all possible combinations between `id`'s of both tables. Note that cross joins do **not** use `ON` or `USING`.
<p></p><img src="images/cross_join.png"><p></p>

In [ ]:
# example 1
SELECT c.name AS city, l.name AS language
FROM cities AS c
CROSS JOIN languages AS l
WHERE c.name LIKE 'Hyder%';

## CASE

`CASE` allows for multiple logical statements to be entered in a simplified way. You can use `CASE` with `WHEN`, `THEN`, `ELSE`, and `END` to define a new goruping field.

In [ ]:
# example 1
SELECT name, continent, code, surface_area,
    CASE WHEN surface_area > 2000000
            THEN 'large'
       WHEN surface_area > 350000
            THEN 'medium'
       ELSE 'small' END
       AS geosize_group
FROM countries;

# example 2


## INTO

`INTO` allows placement of query results into a new table.

In [ ]:
# example 1
SELECT name, continent, code, surface_area,
    CASE WHEN surface_area > 2000000
            THEN 'large'
       WHEN surface_area > 350000
            THEN 'medium'
       ELSE 'small' END
       AS geosize_group
INTO countries_plus
FROM countries;

## Set Theory

In SQL, set theory is needed to fully understand the `UNION`, `UNION ALL`, `INTERSECT`, and `EXCEPT` keywords.

A detailed explanation beginning from the mathematical foundations can be found here: <br>
https://www.sqlshack.com/mathematics-sql-server-fast-introduction-set-theory/

<p></p><img src="images/set_theory.png"><p></p>

Quick breakdown:
- `UNION` - includes every record in both tables, but **does not** double count records in both tables
- `UNION ALL` - includes every record in both tables, but **does** double count records in both tables
- `INTERSECT` - includes every record found in both tables
- `EXCEPT` - includes every record found in one table, but not the other

### UNION

`UNION` includes every record found in both tables, but does not double count records found in both tables. `UNION` can also be used to determine all occurrences of a field across multiple tables, see example 2. You can think of this as a `SELECT DISTINCT` for multiple tables.

<p></p><img src="images/union.png" width="150"><p></p>

In [ ]:
# example 1
SELECT *
FROM economies2010
UNION
SELECT *
FROM economies2015
ORDER BY code, year;

# example 2
SELECT code
FROM currencies
UNION 
SELECT country_code
FROM cities
ORDER BY code;

### UNION ALL

`UNION ALL` includes every record found in both tables and does double count records found in both tables.

<p></p><img src="images/union_all.png" width="150"><p></p>

In [ ]:
# example 1
SELECT country_code AS code, year
FROM populations
UNION ALL
SELECT code, year
FROM economies
ORDER BY code, year;

### INTERSECT

`INTERSECT` includes every record found in both tables. Note that if you select multiple columns, all columns will be evaluated together.
<p></p><img src="images/intersect.png" width="150"><p></p>

In [ ]:
# example 1
SELECT code, year
FROM economies
INTERSECT
SELECT country_code, year
FROM populations
ORDER BY code, year;

# example 2 - which countries also have a city 
#             with the same name as their country?
SELECT name
FROM countries
INTERSECT
SELECT name
FROM cities
ORDER BY name;

### EXCEPT

`EXCEPT` includes every record found in one table but not the other.

<p></p><img src="images/except.png" width="150"><p></p>

In [ ]:
# example 1 - Get the names of cities in cities which are not noted as 
#             capital cities in countries as a single field result
SELECT name
FROM cities
EXCEPT
SELECT capital
FROM countries
ORDER BY name;

# example 2 - reverse of example 1
SELECT capital
FROM countries
EXCEPT
SELECT name
FROM cities
ORDER BY capital;

# for fun, note that example 1 showed 170 rows while example 2 showed
# 136 rows - the tables contained 236 of the world's most populous cities

### Semi-joins and Anti-joins

Semi-joins and Anti-joins function similarly to `WHERE` clauses dependent on values in a second table. They are useful for filtering one tables records based on another tables records. This section will also touch on subqueries.

Anti-joins are particularly useful in identifying which records are causing an incorrect number of records to appear in join queries.

<p></p><img src="images/semi_anti_joins.png" width="400"><p></p>

In [ ]:
# example 1 - semi-join
SELECT DISTINCT name
FROM languages
WHERE code IN
    (SELECT code
    FROM countries
    WHERE region = 'Middle East')
ORDER BY name;

# example 2 - anti-join
SELECT code, name
FROM countries
WHERE continent = 'Oceania'
    AND code NOT IN
    (SELECT code
    FROM currencies);

### Complex Example

This example accomplishes the following:
- Identify the country codes that are included in either economies or currencies but not in populations
- Use this result to determine the names of cities in the countries that match the specification in the previous instruction

In [ ]:
# select the city name
SELECT name
# alias the table where city name resides
FROM cities AS c1
# choose only records matching the result of multiple set theory clauses
WHERE country_code IN
(
    # select appropriate field from economies AS e
    SELECT e.code
    FROM economies AS e
    # get all additional (unique) values of the field from currencies AS c2  
    UNION
    SELECT c2.code
    FROM currencies AS c2
    # exclude those appearing in populations AS p
    EXCEPT
    SELECT p.country_code
    FROM populations AS p
);

## Subqueries

Subqueries are most commonly found inside `WHERE` clauses. For instance, you may need to perform a `GROUP BY` and `AVG` query to determine a threshold used to filter in the `WHERE` clause of your primary query.

Subqueries can also be found in `SELECT` clauses. Note that when placed within a `SELECT` clause, the subquery must be given an alias. A subquery used in a `SELECT` clause is essentially a new calculated column.

Subqueries can also be found in `FROM` clauses. 

### Within `WHERE` clauses

In [ ]:
# example 1
SELECT *
FROM populations
WHERE life_expectancy > 
    1.15 * (SELECT AVG(life_expectancy)
    FROM populations
    WHERE year = 2015)
AND year = 2015;

# example 2 
# select the urban area population for only capital cities
SELECT name, country_code, urbanarea_pop
FROM cities
WHERE name IN
  (SELECT capital
   FROM countries)
ORDER BY urbanarea_pop DESC;

### Within `SELECT` clauses

In [ ]:
# example 1
SELECT countries.name AS country,
  (SELECT COUNT(*)
   FROM cities
   WHERE countries.code = cities.country_code) AS cities_num
FROM countries
ORDER BY cities_num DESC, country
LIMIT 9;

# example 2 - same result as example 1, except using JOIN
SELECT countries.name AS country, COUNT(*) AS cities_num
FROM cities
INNER JOIN countries
ON countries.code = cities.country_code
GROUP BY country
ORDER BY cities_num DESC, country
LIMIT 9;

### Within `FROM` clauses

In [ ]:
# example 1
    # determine the number of languages spoken for reach country, 
    # identified by the country's local name
SELECT local_name, subquery.lang_num
FROM countries,
    (SELECT code, COUNT(name) AS lang_num
    FROM languages
    GROUP BY code) AS subquery
WHERE countries.code = subquery.code
ORDER BY lang_num DESC;

# example 2


### Complex Subquery

The following answers which country had the maximum inflation rate and how high it was.

In [ ]:
# example 1
SELECT name, continent, inflation_rate
FROM countries
INNER JOIN economies
ON countries.code = economies.code
WHERE year = 2015
    AND inflation_rate IN (
        SELECT MAX(inflation_rate) AS max_inf
        FROM (
             SELECT name, continent, inflation_rate
             FROM countries
             INNER JOIN economies
             ON countries.code = economies.code
             WHERE year = 2015) AS subquery
        GROUP BY continent);